# **Inference Step Result Analysis**

In this notebook, the raw inference energy consumption data, with different configurations of inference steps, is analyzed. The results are averaged over the five runs, along with the computation of the standard deviation for each tracked parameter.

In [1]:
# import required libraries
import pandas as pd
import glob
import os
import re

In [ ]:
# get working directory, necessary to gather the data to be analyzed
current_dir = os.getcwd()
print(f"Current Working Directory: {current_dir}")
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
print(f"Parent Directory: {parent_dir}")

## **Helper Functions**
Useful functions to properly format labels and to obtain final results from raw inference energy consumption data.

In [3]:
def get_emission_data(path, model, runs=5, option="summary"):
    """
    Analyze emission experiment results for diffusion models using CodeCarbon logs.

    Processes CodeCarbon CSV output files to generate statistical summaries
    of energy consumption and emissions across multiple experiment runs. It calculates
    mean and standard deviation for energy and emissions metrics, provides output options.

    Parameters:
    -----------
    path : list[str]
        List of file paths to CodeCarbon log CSV files for the experiment.
        Each path should point to a CSV file containing emission metrics.

    model : str
        Name of the model being analyzed. Used for labeling output files and results.

    runs : int, optional (default=1)
        Number of experimental runs to process.
        Recommendation: Use at least 10 runs to reduce statistical variability.
        If more paths are provided than runs, only the first 'runs' will be processed.

    option : str, optional (default="summary")
        Specifies the type of metrics to analyze and output.
        Valid options include:

        - "power": Component power consumption
        - "energy": Energy consumption per component
        - "energy_consumed": Total energy used
        - "emissions": Carbon emissions
        - "emissions_rate": Emission rate
        - "summary": Overview of all metrics

    Returns:
    --------
    None
        Outputs are written to CSV files and printed to the terminal.

    Outputs:
    --------
    - CSV files with mean and standard deviation of selected metrics
    - Formatted table of results printed to the terminal
    - Files saved in the same directory as the input logs,
      with names following the pattern: {model}-{metric}-{type}.csv (e.g., model-emissions-mean.csv)
    """

    # Initialize the summary flag to False. This controls whether a comprehensive summary
    # of all metrics is generated or if the analysis focuses on a specific metric option.
    summary = False
    # Dictionary mapping emission metrics to their visualization and labeling
    # Each entry contains: [column_name, color_set, display_title, unit, description]
    emissdict = {"power": ["power_type", "Set1", "'Component Power'", "[W]", "Power delivered to components"],
                 "energy": ["energy_type", "Set1", "'Energy Consumption per Component'", "[kWh]",
                            "Energy used by components"],
                 "energy_consumed": ["energy_consumed", "rocket_r", "'Total Energy Used'", "[kWh]", "Energy Used"],
                 "emissions": ["emissions", "crest", "'Emissions'", "[CO₂eq]", "'Emissions in CO₂-equivalents, in kg'"],
                 "emissions_rate": ["emissions_rate", "crest", "'Emission Rate'", "[Kg/s]",
                                    "Emissions divided per duration"]}

    # Aggregate data from multiple experimental runs
    all_data = []
    for i, file_path in enumerate(path):
        if i >= runs:
            break
        df = pd.read_csv(file_path)
        all_data.append(df)
    # Concatenate data from all runs into a single dataframe
    dfr = pd.concat(all_data, ignore_index=True)
    # Extract number of inference steps from project name
    dfr['project_name'] = dfr['project_name'].apply(lambda p: re.search(r'(\d+)-steps', p).group(1))

    separator = "-" * 80
    print(separator)
    print(model.center(80))
    print(separator)

    # Iterate through different emission metric options
    for i, k in enumerate(emissdict.keys()):
        df=dfr
        # Prepare mean and standard deviation DataFrames
        column_names = df.columns[5:14]
        df["project_name"] = pd.to_numeric(df["project_name"], errors='coerce')
        # Calculate mean and standard deviation
        deviations = (df.groupby("project_name")[column_names].std()
                      .sort_values(by='project_name', ascending=True).reset_index())
        df = (df.groupby("project_name")[column_names].mean()
              .sort_values(by='project_name', ascending=True).reset_index())
        # Convert project names to strings
        df["project_name"] = df["project_name"].astype(str)
        deviations["project_name"] = deviations["project_name"].astype(str)

        # Handle the summary option: save complete mean and standard deviation files
        if option == "summary":
            summary = True
            df.to_csv(current_dir + fr"\results\inference_steps\{model}\{model}-mean.csv", index=False)
            deviations.to_csv(current_dir + fr"\results\inference_steps\{model}\{model}-std.csv", index=False)

        # Skip iterations if not in summary mode and not first iteration
        if not summary and i != 0:
            continue
        elif summary:
            option = k

        # Handling for power and energy metrics (group values for the same inference steps)
        if option in ("power", "energy"):
            df = melt_and_map(df, option)
            deviations = melt_and_map(deviations, option)

        # Metric title
        print(emissdict[option][2])
        if option in ("power", "energy"):
            df_pivot = format_powen(df, emissdict, option)
            deviations_pivot = format_powen(deviations, emissdict, option)
            print(format_powen(df, emissdict, option, deviations, printed=True).to_string(index=False) + "\n \n")
            # Save results if not in summary mode
            if not summary:
                df_pivot.to_csv(current_dir + fr"\results\inference_steps\{model}\{model}-{option}-mean.csv", index=False)
                deviations_pivot.to_csv(current_dir + fr"\results\inference_steps\{model}\{model}-{option}-std.csv", index=False)
        else:
            # Format and print results tables
            df_pivot = format_rest(df, emissdict, option)
            deviations_pivot = format_rest(deviations, emissdict, option)
            print(format_rest(df, emissdict, option, deviations, printed=True).to_string(index=False) + "\n \n")
            # Save metric-specific mean and standard deviation files
            if not summary:
                df_pivot.to_csv(current_dir + fr"\results\inference_steps\{model}\{model}-{option}-mean.csv", index=False)
                deviations_pivot.to_csv(current_dir + fr"\results\inference_steps\{model}\{model}-{option}-std.csv", index=False)

In [4]:
def melt_and_map(df, option):
    """
    Transforms a DataFrame by reshaping and labeling component-specific energy or power metrics.
    - Reshapes the DataFrame from wide to long format, focusing on CPU, GPU, and RAM metrics
    - Replaces generic column names with component descriptions

    Parameters:
    -----------
    df : pandas.DataFrame
        Input DataFrame containing component-specific metrics. Expected to have columns for CPU, GPU, and RAM metrics.

    option : str
        The type of metric being processed. Either "power" or "energy".

    Returns:
    --------
    pandas.DataFrame
        Reshaped and relabeled DataFrame with the following changes:
        - Converted from wide to long format
        - Component columns renamed to specific hardware descriptions
        - Maintains the original project name as an identifier
    """
    # Reshape the DataFrame from wide to long format, focusing on component-specific metrics
    df = pd.melt(df, id_vars=["project_name"],
                 value_vars=[f"cpu_{option}", f"gpu_{option}", f"ram_{option}"],
                 var_name=f"{option}_type", value_name=f"{option}")

    # Map generic column names to specific hardware component descriptions
    df[f"{option}_type"] = df[f"{option}_type"].map({
        f"cpu_{option}": "CPU (AMD EPYC 7313)",
        f"gpu_{option}": "GPU (NVIDIA A40)",
        f"ram_{option}": "RAM (64 GB)"
    })
    return df

In [5]:
def format_powen(df, emissdict, option, std=pd.DataFrame(), printed=False):
    """
    Formats DataFrame of performance metrics with sorted project names and units appended to numeric values.
    Works with either "energy" or "power" options.

    Parameters:
    -----------
    df : pandas.DataFrame
        Input DataFrame containing performance and energy metrics.
        Expected to have columns 'project_name' and metric-specific columns.

    emissdict : dict
        Dictionary mapping metric options to specific configuration details.

    option : str
        Specifies the type of metric to be formatted. Must be a valid key in emissdict.

    std : pandas.DataFrame
        DataFrame containing the confidence intervals for the obtained results

    printed : bool
        Flag to check whether the function is being used for printing the results to terminal.

    Returns:
    --------
    pandas.DataFrame
        Formatted DataFrame with:
        - Rows sorted by numeric project identifier
        - Columns: 'Inference steps', 'CPU', 'GPU', 'RAM'
        - Numeric values with appropriate units
    """
    # Handles case for printing to terminal, adds the confidence interval to the data visualization
    if printed:
        # Pivot the DataFrame to structure it with 'project_name' as rows and a metric as columns
        df_pivot = df.pivot(index="project_name", columns=emissdict[option][0], values=f"{option}").reset_index()
        std_pivot = std.pivot(index="project_name", columns=emissdict[option][0], values=f"{option}").reset_index()
        # Sort the DataFrame by extracting numeric identifiers from the 'project_name' column
        df_pivot = df_pivot.loc[df_pivot['project_name'].str.extract(r'(\d+)').astype(int).squeeze().sort_values().index]
        std_pivot = std_pivot.loc[std_pivot['project_name'].str.extract(r'(\d+)').astype(int).squeeze().sort_values().index]
        # Rename columns
        df_pivot.columns = ['Inference steps', 'CPU (AMD EPYC 7313)', 'GPU (NVIDIA A40)', 'RAM (64 GB)']
        std_pivot.columns = ['Inference steps', 'CPU (AMD EPYC 7313)', 'GPU (NVIDIA A40)', 'RAM (64 GB)']
        # Append units to numeric values in 'CPU', 'GPU', and 'RAM' columns
        for col in ['CPU (AMD EPYC 7313)', 'GPU (NVIDIA A40)', 'RAM (64 GB)']:
            df_values = df_pivot[col]
            std_values = std_pivot[col]
            # Round both sets of values
            rounded_df_values = round(df_values, 8)
            df_str = rounded_df_values.astype(str)
            # Add confidence interval
            std_str = std_values.apply(lambda b: "{:.1e}".format(b))
            combined_data = df_str + " ± " + std_str + f" {emissdict[option][3]}"
            df_pivot[col] = combined_data
    else:
        # Pivot the DataFrame to structure it with 'project_name' as rows and a metric as columns
        df_pivot = df.pivot(index="project_name", columns=emissdict[option][0], values=f"{option}").reset_index()
        # Sort the DataFrame by extracting numeric identifiers from the 'project_name' column
        df_pivot = df_pivot.loc[df_pivot['project_name'].str.extract(r'(\d+)').astype(int).squeeze().sort_values().index]
        # Rename columns
        df_pivot.columns = ['Inference steps', 'CPU (AMD EPYC 7313)', 'GPU (NVIDIA A40)', 'RAM (64 GB)']
        # Append units to numeric values in 'CPU', 'GPU', and 'RAM' columns
        for col in ['CPU (AMD EPYC 7313)', 'GPU (NVIDIA A40)', 'RAM (64 GB)']:
            df_pivot[col] = df_pivot[col].astype(str) + f" {emissdict[option][3]}"
    return df_pivot

In [6]:
def format_rest(df, emissdict, option, std=pd.DataFrame(), printed=False):
    """
    Formats a DataFrame by standardizing column names and adding units to metric values.

    - Appends unit notation to the metric values
    - Renames columns to provide clear headers for printing

    Parameters:
    -----------
    df : pandas.DataFrame
        Input DataFrame containing emission or energy metrics.

    emissdict : dict
        A dictionary containing metadata for different emission metrics.

    option : str
        The specific metric being processed. Must be a key present in the emissdict dictionary.

    std : pandas.DataFrame
        DataFrame containing the confidence intervals for the obtained results

    printed : bool
        Flag to check whether the function is being used for printing the results to terminal.

    Returns:
    --------
    pandas.DataFrame
        Formatted DataFrame with:
        - Metric values annotated with their units
        - Columns renamed to provide clearer headers
    """
    # Handles case for printing to terminal, adds the confidence interval to the data visualization
    if printed:
        std_values = std[f'{option}']
        # Round both sets of values
        rounded_df_values = round(df[f'{option}'], 8)
        df_str = rounded_df_values.astype(str)
        std_str = std_values.apply(lambda m: "{:.1e}".format(m))
        combined_data = df_str + " ± " + std_str + f" {emissdict[option][3]}"
        df[f'{option} '] = combined_data
        # Create a table with project name and annotated metric
        table = df[['project_name', f'{option} ']]
        # Rename columns to descriptive headers
        table.columns = ['Inference steps', f'{emissdict[option][2]}']
        return table
    else:
        # Append unit notation to the metric values as a string
        df[f'{option} '] = df[f'{option}'].astype(str) + f" {emissdict[option][3]}"
        # Create a table with project name and annotated metric
        table = df[['project_name', f'{option} ']]
        # Rename columns to descriptive headers
        table.columns = ['Inference steps', f'{emissdict[option][2]}']
        return table

## **Data analysis**
Using the previously defined functions, we now can analyze the data obtained for the five different runs for all considered models.

In [7]:
# List of all models for analysis
model_list = ["AudioLDM", "AudioLDM2", "Make-an-Audio", "Make-an-Audio-2", "Stable Audio Open", "Tango", "Tango2"]

In [8]:
# analyze all models
for x in model_list:
        # Analyze emission data for the current model
        csvs = glob.glob(fr'{current_dir}\results\inference_steps\{x}\{x}-emissions-run*')
        get_emission_data(csvs, option="summary", model=f'{x}')

--------------------------------------------------------------------------------
                                    AudioLDM                                    
--------------------------------------------------------------------------------
'Component Power'
Inference steps        CPU (AMD EPYC 7313)           GPU (NVIDIA A40)              RAM (64 GB)
             10 174.17448722 ± 2.0e+00 [W] 191.70863257 ± 2.0e+02 [W] 0.64875984 ± 4.1e-04 [W]
             25 178.11948155 ± 1.2e+01 [W] 263.68415916 ± 1.7e+02 [W] 0.64890633 ± 8.2e-05 [W]
             50 183.54889682 ± 8.2e+00 [W] 286.14782324 ± 2.0e+02 [W] 0.64894295 ± 0.0e+00 [W]
            100 184.84487794 ± 1.5e+01 [W] 247.33431273 ± 1.6e+02 [W] 0.64894295 ± 0.0e+00 [W]
            150 180.81419036 ± 1.5e+01 [W] 233.02426037 ± 2.1e+02 [W] 0.64894295 ± 0.0e+00 [W]
            200 191.77383892 ± 1.1e+01 [W] 120.74843237 ± 1.7e+02 [W] 0.64894295 ± 0.0e+00 [W]
 

'Energy Consumption per Component'
Inference steps        CPU (AMD EPYC

In [9]:
# analyze single models
model_name = "AudioLDM" # model = "AudioLDM2" # model = "Make-an-Audio" # . . .
csvs = glob.glob(fr'{current_dir}\results\inference_steps\{model_name}\{model_name}-emissions-run*')
get_emission_data(csvs, option="summary", model= model_name)

--------------------------------------------------------------------------------
                                    AudioLDM                                    
--------------------------------------------------------------------------------
'Component Power'
Inference steps        CPU (AMD EPYC 7313)           GPU (NVIDIA A40)              RAM (64 GB)
             10 174.17448722 ± 2.0e+00 [W] 191.70863257 ± 2.0e+02 [W] 0.64875984 ± 4.1e-04 [W]
             25 178.11948155 ± 1.2e+01 [W] 263.68415916 ± 1.7e+02 [W] 0.64890633 ± 8.2e-05 [W]
             50 183.54889682 ± 8.2e+00 [W] 286.14782324 ± 2.0e+02 [W] 0.64894295 ± 0.0e+00 [W]
            100 184.84487794 ± 1.5e+01 [W] 247.33431273 ± 1.6e+02 [W] 0.64894295 ± 0.0e+00 [W]
            150 180.81419036 ± 1.5e+01 [W] 233.02426037 ± 2.1e+02 [W] 0.64894295 ± 0.0e+00 [W]
            200 191.77383892 ± 1.1e+01 [W] 120.74843237 ± 1.7e+02 [W] 0.64894295 ± 0.0e+00 [W]
 

'Energy Consumption per Component'
Inference steps        CPU (AMD EPYC